In [197]:
%%bash
wget -q http://cys.bios.niu.edu/dbCAN2/download/Databases/CAZyDB.07312018.pr-with-ec.txt
wget -q http://cys.bios.niu.edu/dbCAN2/download/Databases/dbCAN-HMMdb-V7.txt

  HTTP/1.1 200 OK
  Date: Wed, 17 Oct 2018 18:28:20 GMT
  Server: Apache/2.4.27 (Ubuntu)
  Last-Modified: Thu, 13 Sep 2018 20:14:48 GMT
  ETag: "3fa33-575c65bd8079b"
  Accept-Ranges: bytes
  Content-Length: 260659
  Vary: Accept-Encoding
  Keep-Alive: timeout=5, max=100
  Connection: Keep-Alive
  Content-Type: text/plain
  HTTP/1.1 200 OK
  Date: Wed, 17 Oct 2018 18:28:20 GMT
  Server: Apache/2.4.27 (Ubuntu)
  Last-Modified: Sat, 25 Aug 2018 02:53:55 GMT
  ETag: "539c8d8-574399a638a5a"
  Accept-Ranges: bytes
  Content-Length: 87673048
  Vary: Accept-Encoding
  Keep-Alive: timeout=5, max=100
  Connection: Keep-Alive
  Content-Type: text/plain


In [198]:
#         AAB42155.1      CBM2|CBM3|GH9|3.2.1.4
#         BAD44715.1      GH18|3.2.1.14
#         ABE98421.1      GT11|2.4.1.69
from collections import defaultdict

ec_map = defaultdict(set)
with open("CAZyDB.07312018.pr-with-ec.txt") as fh:
    for line in fh:
        toks = line.strip().split()
        attrs = toks[-1].split("|")
        ecs = []
        hmm_ids = []
        for attr in attrs:
            if attr[0].isdigit():
                ecs.append(attr)
            else:
                hmm_ids.append(attr)
#                 if "_" in attr:
#                     hmm_ids.append(attr.partition("_")[0])
        for hmm_id in hmm_ids:
            for ec in ecs:
                ec_map[hmm_id].update([ec])

hmm_ids = list(ec_map.keys())
for hmm_id in hmm_ids:
    if not "_" in hmm_id:
        continue
    parent_hmm = hmm_id.partition("_")[0]
    if parent_hmm in ec_map:
        continue
    ec_map[parent_hmm] = ec_map[parent_hmm].union(ec_map[hmm_id])

hmm_ids = list(ec_map.keys())
# reducing the EC set by removing lowest level where needed
for hmm_id in hmm_ids:
    current_ec_set = ec_map[hmm_id]
#     if len(current_ec_set) < 4:
#         continue
    new_ec_set = set()
    for ec in current_ec_set:
        primary = ec.rpartition(".")[0]
        up_one = f"{primary}.-"
        if up_one in current_ec_set:
            new_ec_set.update([up_one])
            continue
        count = len([i for i in current_ec_set if i.startswith(primary)])
        # these three levels are observed more than twice
        if count > 2:
            new_ec_set.update([up_one])
        else:
            new_ec_set.update([ec])
    # update the main reference
    ec_map[hmm_id] = new_ec_set


In [199]:
with open("dbCAN-HMMdb-V7.txt") as infile, open("dbCAN-HMMdb-V7-ACC.txt", "w") as outfile:
    for line in infile:
        if line.startswith("DESC") or line.startswith("ACC"):
            continue
        line = line.strip("\r\n")
        if line.startswith("NAME"):
            model_name = line.split()[-1]
            hmm_name = line.split()[-1].partition(".")[0]
            family = hmm_name.partition("_")[0]
            if hmm_name in ec_map:
                desc = ",".join(ec_map[hmm_name])
            else:
                desc = ""
            print(line, file=outfile)
#             print(f"DESC  {desc}~~~{family}~~~{hmm_name}", file=outfile)
            print(f"ACC   {desc}~~~{family}~~~{hmm_name}~~~{model_name}", file=outfile)
        else:
            print(line, file=outfile)

In [200]:
%%bash
hmmconvert -b dbCAN-HMMdb-V7-ACC.txt > dbCAN-HMMdb-V7-ACC.hmm
hmmpress -f dbCAN-HMMdb-V7-ACC.hmm

Working...    done.
Pressed and indexed 607 HMMs (607 names and 607 accessions).
Models pressed into binary file:   dbCAN-HMMdb-V7-ACC.hmm.h3m
SSI index for binary model file:   dbCAN-HMMdb-V7-ACC.hmm.h3i
Profiles (MSV part) pressed into:  dbCAN-HMMdb-V7-ACC.hmm.h3f
Profiles (remainder) pressed into: dbCAN-HMMdb-V7-ACC.hmm.h3p


In [201]:
%%bash
wget -q https://raw.githubusercontent.com/brwnj/prokka/update-hamap/bin/prokka-hamap_to_hmm
wget -q ftp://ftp.expasy.org/databases/hamap/hamap_seed_alignments.tar.gz
wget -q ftp://ftp.expasy.org/databases/hamap/hamap_rules.dat
wget -q ftp://ftp.expasy.org/databases/hamap/rules_index.dat

  HTTP/1.1 200 OK
  Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
  Strict-Transport-Security: max-age=31536000
  X-Content-Type-Options: nosniff
  X-Frame-Options: deny
  X-XSS-Protection: 1; mode=block
  ETag: "8507bc75a3269fe59bbfc83543207eeef523d672"
  Content-Type: text/plain; charset=utf-8
  Cache-Control: max-age=300
  X-Geo-Block-List:
  X-GitHub-Request-Id: 69A8:5E6B:102716:11290A:5BC77F85
  Content-Length: 4290
  Accept-Ranges: bytes
  Date: Wed, 17 Oct 2018 18:29:25 GMT
  Via: 1.1 varnish
  Connection: keep-alive
  X-Served-By: cache-dca17736-DCA
  X-Cache: MISS
  X-Cache-Hits: 0
  X-Timer: S1539800965.439979,VS0,VE42
  Vary: Authorization,Accept-Encoding
  Access-Control-Allow-Origin: *
  X-Fastly-Request-ID: 45b332a690ad43fc96a832f6d6587f400adba9bd
  Expires: Wed, 17 Oct 2018 18:34:25 GMT
  Source-Age: 0

--> USER anonymous

--> PASS Turtle Power!

--> SYST

--> PWD

--> TYPE I

--> CWD /databases/hamap

--> PASV

--> LIST -a

drwxrwxr-x  

In [203]:
%%bash
perl prokka-hamap_to_hmm --datadir . 2> /dev/null

In [204]:
# add a unique accession to each HMM
with open("HAMAP.hmm") as infile, open("HAMAP-ACC.txt", "w") as outfile:
    for line in infile:
        if line.startswith("NAME"):
            hmm_name = line.split()[-1]
        if line.startswith("ACC"):
            continue
        line = line.strip("\r\n")
        if line.startswith("DESC"):
            acc = line.replace(" ", "^").replace("DESC^^", "ACC   ")
            acc += f"~~~{hmm_name}"
            print(acc, file=outfile)
        else:
            print(line, file=outfile)

In [205]:
%%bash
hmmconvert -b HAMAP-ACC.txt > HAMAP-ACC.hmm
hmmpress -f HAMAP-ACC.hmm

Working...    done.
Pressed and indexed 1533 HMMs (1533 names and 1533 accessions).
Models pressed into binary file:   HAMAP-ACC.hmm.h3m
SSI index for binary model file:   HAMAP-ACC.hmm.h3i
Profiles (MSV part) pressed into:  HAMAP-ACC.hmm.h3f
Profiles (remainder) pressed into: HAMAP-ACC.hmm.h3p


In [207]:
%%bash
# the hmm
wget -c -q https://ftp.ncbi.nlm.nih.gov/hmm/TIGRFAMs/release_15.0/TIGRFAMs_15.0_HMM.LIB.gz
# the ECs
wget -c -q https://ftp.ncbi.nlm.nih.gov/hmm/TIGRFAMs/release_15.0/TIGRFAMs_15.0_INFO.tar.gz
# gene and product
wget -c -q https://ftp.ncbi.nlm.nih.gov/hmm/TIGRFAMs/TIGRFAMs_improvements_2018-10-11.tsv
# extract INFO archive
mkdir info
tar -xf TIGRFAMs_15.0_INFO.tar.gz --directory info/


--> USER anonymous

--> PASS Turtle Power!

--> SYST

--> PWD

--> TYPE I

--> CWD /pub/data/TIGRFAMs

--> PASV

--> LIST -a

drwxr-xr-x  22 ftp      ftp            33 Sep 17  2014 .
drwxr-xr-x  26 ftp      ftp            26 Jul 23 23:08 ..
drwxrwxr-x   2 ftp      ftp             9 May 15  2012 1.0_Release
drwxrwxr-x   2 ftp      ftp             8 May 15  2012 1.1_Release
drwxrwxr-x   2 ftp      ftp             8 May 15  2012 1.2_Release
drwxr-xr-x   2 ftp      ftp            15 May 15  2012 10.0_Release
drwxr-xr-x   2 ftp      ftp            11 May 15  2012 11.0_Release
drwxr-xr-x   2 ftp      ftp            12 Aug 15  2012 12.0_Release
drwxr-xr-x   2 ftp      ftp            12 Jan 21  2014 13.0_Release
drwxr-xr-x   2 ftp      ftp            11 Sep 17  2014 14.0_Release
drwxrwxr-x   2 ftp      ftp            11 May 15  2012 2.0_Release
drwxrwxr-x   2 ftp      ftp            10 May 15  2012 2.1_Release
drwxrwxr-x   2 ftp      ftp            13 May 15  2012 3.0_Release
drwxrwxr-x   2 f

In [244]:
import os


# get the ECs for TIGRFAMs
ec_map = dict()
gene_map = dict()
for info_file in os.listdir("info"):
    with open(os.path.join("info", info_file), mode="r", encoding="ISO-8859-1") as fh:
        hmm_id = ""
        ec = ""
        product = ""
        name = ""
        for line in fh:
            if line.startswith("AC"):
                hmm_id = line.strip("\r\n").partition("  ")[-1]
            elif line.startswith("EC"):
                ec = line.strip("\r\n").partition("  ")[-1]
            # desc/product def
            elif line.startswith("DE"):
                product = line.strip("\r\n").partition("  ")[-1].replace(" ", "^")
            # gene symbol
            elif line.startswith("GS"):
                name = line.strip("\r\n").partition("  ")[-1]
        ec_map[hmm_id] = ec.replace(" ", ",")
        gene_map[hmm_id] = dict(name=name, product=product)

In [245]:
import csv


# update the gene map
with open("TIGRFAMs_improvements_2018-10-11.tsv") as fh:
    reader = csv.DictReader(fh, delimiter="\t")
    for row in reader:
        gene_map[row["# accession"]] = dict(
            name=row["genesymbol"].replace("NULL", ""), 
            product=row["product_name"].replace(" ", "^")
        )

In [246]:
gene_map["TIGR00005"]

{'name': '', 'product': 'RluA^family^pseudouridine^synthase'}

In [247]:
import gzip


with gzip.open("TIGRFAMs_15.0_HMM.LIB.gz", "rt") as infile, open("TIGRFAMs_15_0_ACC.txt", "w") as outfile:
    for line in infile:
        if line.startswith("NAME"):
            hmm_name = line.split()[-1]
        if line.startswith("DESC"):
            continue
        line = line.strip("\r\n")
        if line.startswith("ACC"):
            # ec, gene, product, HMM ID
            try:
                ec = ec_map[hmm_name]
            except KeyError:
                ec = ""
            gene = gene_map[hmm_name]
            acc = f"ACC   {ec}~~~{gene['name']}~~~{gene['product']}~~~{hmm_name}"
            check = "{ec}~~~{gene['name']}~~~{gene['product']}~~~{hmm_name}"
            print(acc, file=outfile)
        else:
            print(line, file=outfile)

In [248]:
%%bash
hmmconvert -b TIGRFAMs_15_0_ACC.txt > TIGRFAMs_15_0_ACC.hmm
hmmpress -f TIGRFAMs_15_0_ACC.hmm

Working...    done.
Pressed and indexed 4488 HMMs (4488 names and 4488 accessions).
Models pressed into binary file:   TIGRFAMs_15_0_ACC.hmm.h3m
SSI index for binary model file:   TIGRFAMs_15_0_ACC.hmm.h3i
Profiles (MSV part) pressed into:  TIGRFAMs_15_0_ACC.hmm.h3f
Profiles (remainder) pressed into: TIGRFAMs_15_0_ACC.hmm.h3p
